In [2]:
import SCons as sc

ModuleNotFoundError: No module named 'SCons'

In [1]:
import os

In [3]:
from  cffi import FFI

In [4]:
import subprocess

In [6]:
subprocess.run(["dir"],capture_output=True, shell=True)

CompletedProcess(args=['dir'], returncode=0, stdout=b'__temp__test.cpp  a.out  test.cpp  test.ipynb  test.so\n', stderr=b'')

0

In [16]:
import numpy as np

In [10]:
import ctypes

In [67]:
a = ctypes.CDLL("./libtest.dll")

In [1]:
filename = "test.cpp"

tempname = "__temp__"+filename

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


In [2]:
with  open(filename) as cppsource:
    header = []
    functions = []
    header_line = 0
    
    for i, line in enumerate(cppsource):
        if line[:16] == "//@extern_python":
            header_line = i
        if line[:13] == "//@python_def":
            functions.append(")".join(cppsource.readline().split(")")[:-1])+");\r\n")
        if line[:13] == "//@python_var":
            functions.append(cppsource.readline())
        if line[:14] == "//@python_defs":
            while line[:18] == "//@python_defs_end":
                line = cppsource.readline()
                header.append(cppsource.readline())
            


In [3]:
with open(tempname,"w") as cpp_source:
    with open(filename) as cppsource:
        for i,line in enumerate(cppsource):
            if i == header_line:
                cpp_source.write('extern "C"{\r\n')
                for cdefs in header:
                    cpp_source.write("__declspec(dllexport) "+cdefs)
                cpp_source.write('};\r\n')
            else:
                cpp_source.write(line)
                

In [4]:
if os.system(f'clang -o libtest.dll {tempname} -shared') == 2:
    raise SyntaxError("compilation failed.")

In [15]:
from tqdm import tqdm 

In [25]:
import numpy as np
a = np.ctypeslib.ndpointer()

In [28]:
a = np.ctypeslib.ndpointer(dtype = np.int32)

In [30]:
vars(a)

mappingproxy({'_dtype_': dtype('int32'),
              '_shape_': None,
              '_ndim_': None,
              '_flags_': None,
              '__module__': 'numpy.ctypeslib',
              '__doc__': None})

In [225]:
def cdll_types(header):
    ans = {}
    for line in header:
        if line.lstrip():
            if len(line.split("("))<2: continue
            func_name = line.split("(")[0].split()[-2:]
            #print(func_name)
            restype     = (func_name[0])
            name        = func_name[1]
            func_args   = line.split("(")[1].split(")")[0].split(",")
            argtypes    = []
            argnames    = []
            for func_arg in func_args:
                args = func_arg.split()
                if args[1][0] == "*" or args[1][0] == "&":
                    argtypes.append(args[0]+args[1][0])
                    argnames.append(args[1][1:])
                else:
                    argtypes.append(args[0])
                    argnames.append(args[1])

            ans[name] = (argtypes, argnames, restype)
    return ans
                

def c_interface(_cdll, func_name, argtypes, restype = None):
    func = eval(f'_cdll.{func_name}')
    func.argtypes = argtypes
    if restype is not None:
        func.restype = restype

def interface_from_header(cdll, header, py_dict, include = None, exclude = None):
    cdll.header = header
    c_funcs = cdll_types(header)
    if exclude is None:
        exclude = []
    for f_name in c_funcs:
        if f_name in exclude: continue
        if include is not None and not f_name in include: continue
        args, anames,  res = c_funcs[f_name]
        args = [py_dict[arg] for arg in args]
        res  = py_dict[res]
        c_interface(cdll, f_name, args, res)

def function_make(cdll, fname, argtypes, args, restype):
    argstring = ','.join(args)
    convert = ""
    for name, types in zip(args, argtypes):
        if hasattr(c_to_py[types],"_dtype_"):
            convert+=f"\t\tif not isinstance({name}, np.ndarray):\n"
            convert+=f"\t\t\t{name}=np.array({name}, dtype=np.{c_to_py[types]._dtype_})\n"
    
    pdef = f"def closure(cdll):\n"
    pdef += f"\tdef {fname}({argstring}):\n"
    if convert:
        pdef += convert
    #pdef += f"\t\tprint(c, a, b, n)\n"
    pdef += f"\t\tcdll.{fname}({argstring})\n"
    pdef += f"\treturn {fname}"
    print(pdef)
    exec(pdef)
    return locals()['closure'](cdll)

In [172]:
# c and python interface.
c_to_py ={
    'int'       : ctypes.c_int,
    'float'     : ctypes.c_float,
    'float&'     : ctypes.c_float,
    'double'    : ctypes.c_double,
    'float*'    : np.ctypeslib.ndpointer(dtype=np.float32),
    'int*'      : np.ctypeslib.ndpointer(dtype=np.int32),
    'double*'   : np.ctypeslib.ndpointer(dtype=np.float64),
    'void'      : None
}


In [187]:
types = cdll_types(header)

In [206]:
interface_from_header(a, header, c_to_py)

In [121]:
a

<CDLL './libtest.dll', handle 7ffdf4130000 at 0x28382a6df08>

In [140]:
header

['int add(int a, int b);\r\n',
 'void array_add(int* c, int* a, int* b, int n);\r\n']

In [170]:
closure

NameError: name 'closure' is not defined

In [226]:
b = function_make(a, 'array_add', *types['array_add'])

def closure(cdll):
	def array_add(c,a,b,n):
		if not isinstance(c, np.ndarray):
			c=np.array(c, dtype=np.int32)
		if not isinstance(a, np.ndarray):
			a=np.array(a, dtype=np.int32)
		if not isinstance(b, np.ndarray):
			b=np.array(b, dtype=np.int32)
		print(c, a, b, n)
		cdll.array_add(c,a,b,n)
	return array_add


In [227]:
b(ttt, [3], [2], 1)

[3] [3] [2] 1


In [228]:
ttt

array([5])

In [220]:
ttt1 = np.array(ttt)

In [221]:
a.array_add(ttt1, np.array([3]),np.array([2]),1)

1

In [222]:
ttt

array([3])

In [217]:
ttt[0] =3

In [211]:
ttt

array([3])

In [152]:
ttt

array([1634625072])

In [181]:
ttt = np.empty([1], dtype = np.int32)
b(ttt,[3],[2],1)

NameError: name 'np' is not defined

In [168]:
import inspect

In [182]:
print(inspect.getsource(b))

OSError: could not get source code

In [132]:
b(2,3)

TypeError: array_add() missing 2 required positional arguments: 'b' and 'n'

In [21]:
if os.system(f'g++ -o test.so {tempname} -fPIC -shared') == 2:
    raise SyntaxError("compilation failed.")
else:
    os.remove(tempname)

In [22]:
lib = ctypes.CDLL(f'./test.so')

In [23]:
import platform

In [24]:
cpp_header = "".join(header)


c_to_py ={
    'int'       : ctypes.c_int,
    'float'     : ctypes.c_float,
    'float&'     : ctypes.c_float,
    'double'    : ctypes.c_double,
    'float*'    : np.ctypeslib.ndpointer(dtype=np.float32),
    'int*'      : np.ctypeslib.ndpointer(dtype=np.int32),
    'double*'   : np.ctypeslib.ndpointer(dtype=np.float64),
    'void'      : None
}



def cdll_types(header):
    ans = {}
    for line in header.splitlines():
        if line.lstrip():
            if len(line.split("("))<2: continue
            func_name = line.split("(")[0].split()[-2:]
            #print(func_name)
            restype     = (func_name[0])
            name        = func_name[1]
            func_args   = line.split("(")[1].split(")")[0].split(",")
            argtypes    = [(func_arg.split()[0]) for func_arg in func_args]

            ans[name] = (argtypes, restype)
    return ans
                

def c_interface(_cdll, func_name, argtypes, restype = None):
    func = eval(f'_cdll.{func_name}')
    func.argtypes = argtypes
    if restype is not None:
        func.restype = restype

def interface_from_header(cdll, header, py_dict, include = None, exclude = None):
    cdll.header = header
    c_funcs = cdll_types(header)
    if exclude is None:
        exclude = []
    for f_name in c_funcs:
        if f_name in exclude: continue
        if include is not None and not f_name in include: continue
        args, res = c_funcs[f_name]
        args = [py_dict[arg] for arg in args]
        res  = py_dict[res]
        c_interface(cdll, f_name, args, res)

#print(c_ext)
interface_from_header(lib, cpp_header, c_to_py)

In [26]:
a = np.empty([2],dtype = np.int32)
lib.array_add(a, np.array([1,2]).astype(np.int32),np.array([3,4]).astype(np.int32),2)

2

In [27]:
a

array([4, 6], dtype=int32)